In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv
import copy

In [2]:
def prediction(Il,Ildot,I,t):
    
    L = 5
    C = 500e-9
    R = 50e3
    
    A = np.array([[1, t],
                  [-t/(L*C), 1 - (t/(R*C))]])
    
    X = np.array([[Il],
                 [Ildot]])
    
    B = np.array([[0],
                  [t/(L*C)]])
    
    U = I
    
    X_predition = A.dot(X) + B.dot(U)

    
    return X_predition

In [3]:
#time 
T = 0.14

t = np.linspace(0,T,100e3*T)
#initialise inductor current and its derivertive
Il_true = np.zeros(len(t))
Ildot_true = np.zeros(len(t))

#cicuit component definitions
L = 5
C = 500e-9
R = 50e3
#create input signal 
wr = 1/np.sqrt(L*C)

I_true = 25e-6 * np.sin(wr * t)

#time step
dt = t[1] - t[0]

#create data 
I = np.zeros((2,len(t)))

for i in range(1,len(t)):
    a =  prediction(I[0][i-1],I[1][i-1],I_true[i-1],dt).T
    I[:,i] = a

Il_true = I[0]
Ildot_true = I[1]
    
V_true = L*Ildot_true
noise = np.random.normal(loc=0, scale=0.5, size=len(t))
V_meas = V_true + noise

In [4]:
#state transition model 
A = np.array([[1, dt],
              [-dt/(L*C), (1 - (dt/(R*C)))]],dtype='float')

#process noise(model noise)
Q = np.array([[4e-9, 0],
              [0, 4e-9]])

#measurement noise
R = np.array([[4]])

#Initial Predicted (a priori) estimate
P=Q
#store values for plotting
Il_a = np.zeros([len(t)])
Ildot_a = np.zeros([len(t)])

#initial values
Il_a[0]    = 0
Ildot_a[0] = 0

#set the input matrix to be the true input
I_a = I_true

# for i in range(1,len(t)):
for i in range(len(t)):
    #prediction
    X_hat = prediction(Il_a[i-1], Ildot_a[i-1], I_a[i-1],dt)

    #predicted estimate covariance
    P_hat = np.diag(np.diag(A.dot(P).dot(A.T))) + Q

    #observation matrix v(t) = Ldi/dt
    H = np.array([[0,L]])
    
    #innovation covariance
    S = H.dot(P_hat).dot(H.T) + R
    
    # Kalman Gain
    K = P_hat.dot(H.T).dot(inv(S))

    #measurement - prediction 
    y = V_meas[i] - H.dot(X_hat)   

    #updated state estimate
    X = X_hat + K.dot(y)
    
    #Updated (a posteriori) estimate covariance
    P = (np.identity(2) - K.dot(H)).dot(P_hat)
    
    #update values
    Il_a[i]    = X[0]
    Ildot_a[i] = X[1]

In [5]:
plt.figure()
plt.plot(t,Il_a, label = 'Predicted')
plt.plot(t,Il_true,label ='True')
plt.xlabel('Time [s]')
plt.ylabel('$I_L$ [A]')
plt.legend()
plt.tight_layout()
plt.savefig("Inductor_current.png", dpi=300,format='png')

In [6]:
plt.figure()
plt.plot(t,V_meas/L,alpha = 0.5,label = 'Measured')
plt.plot(t,Ildot_a,label = "Predicted")
plt.plot(t,Ildot_true,label ="True")

plt.xlabel('Time [s]')
plt.ylabel('$\dot{I}_{L} [As^{-1}]$')
plt.legend()
plt.savefig("Inductor_current_deriv.png", dpi=300,format='png')

In [7]:
plt.figure()
plt.plot(t,V_meas,alpha = 0.5,label = 'Measured')
plt.plot(t,Ildot_a*L,label = "Predicted")
plt.plot(t,Ildot_true*L,label ="True")

plt.xlabel('Time [s]')
plt.ylabel('Voltage [V]')
plt.legend()
plt.savefig("Inductor_voltage.png", dpi=300,format='png')